In [87]:
from pandas import read_csv, unique

import numpy as np

from scipy.interpolate import interp1d
from scipy.stats import mode

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from tensorflow import stack
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling1D, BatchNormalization, MaxPool1D, Reshape, Activation
from keras.layers import Conv1D, LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [88]:
import tensorflow 
print("GPU support Available:", tensorflow.test.is_built_with_cuda())

GPU support Available: True


In [89]:
def read_data(filepath):
    df = read_csv(filepath, header=None, names=['timestamp', 'X', 'Y', 'Z'], error_bad_lines=False)
    df['Z'] = df['Z'].str.replace(';', '').apply(convert_to_float)
    df.dropna(axis=0, how='any', inplace=True)
    return df

def convert_to_float(x):
    try:
        return np.float64(x)
    except:
        return np.nan


In [90]:
# Import the os module
import os

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Print the type of the returned object
print("os.getcwd() returns an object of type: {0}".format(type(cwd)))


Current working directory: /home/bilz/Documents/GitHub/HAR
os.getcwd() returns an object of type: <class 'str'>


In [91]:
df = read_data(cwd + '/dataset/test_raw_2.txt')
df

,timestamp,X,Y,Z
0,49105962326000,-0.694638,12.680544,0.503953
1,49106062271000,5.012288,11.264028,0.953424
2,49106112167000,4.903325,10.882658,-0.081722
3,49106222305000,-0.612916,18.496431,3.023717
4,49106332290000,-1.184970,12.108489,7.205164
...,...,...,...,...
115,49117992141000,-4.753501,4.208687,11.141444
116,49118252334000,-1.566340,3.759216,3.255263
117,49118362319000,-3.214402,2.955615,0.762740
118,49118472305000,-2.642348,19.572440,8.008764


In [92]:
df['X'] = (df['X']-df['X'].min())/(df['X'].max()-df['X'].min())
df['Y'] = (df['Y']-df['Y'].min())/(df['Y'].max()-df['Y'].min())
df['Z'] = (df['Z']-df['Z'].min())/(df['Z'].max()-df['Z'].min())
df

,timestamp,X,Y,Z
0,49105962326000,0.393145,0.752446,0.436839
1,49106062271000,0.674731,0.701566,0.455203
2,49106112167000,0.669355,0.687867,0.412910
3,49106222305000,0.397177,0.961350,0.539789
4,49106332290000,0.368952,0.731898,0.710629
...,...,...,...,...
115,49117992141000,0.192876,0.448141,0.871452
116,49118252334000,0.350134,0.431996,0.549249
117,49118362319000,0.268817,0.403131,0.447412
118,49118472305000,0.297043,1.000000,0.743461


In [93]:
def segments(df, time_steps, step):
    N_FEATURES = 3
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['X'].values[i:i+time_steps]
        ys = df['Y'].values[i:i+time_steps]
        zs = df['Z'].values[i:i+time_steps]
        
        
        segments.append([xs, ys, zs])
        
    
    reshaped_segments = np.asarray(segments, dtype=np.float32).reshape(-1, time_steps, N_FEATURES)
    
    
    return reshaped_segments

In [94]:
TIME_PERIOD = 80
STEP_DISTANCE = 40
LABEL = 'activityEncode'
x = segments(df, TIME_PERIOD, STEP_DISTANCE)

In [95]:
print(x.shape)

(1, 80, 3)


In [ ]:
## Input and Output Dimensions
time_period, sensors = x.shape[1], x.shape[2]

In [ ]:
## reshaping data
input_shape = time_period * sensors
x = x.reshape(x.shape[0], input_shape)
print("Input Shape: ", input_shape)
print("Input Data Shape: ", x.shape)

In [ ]:
x = x.astype('float32')

In [ ]:
from keras.models import load_model
# load the trained model to a file
model = load_model(cwd +'/models/Bilz_CNN_TrainedModel.h5')
try:
    # load the model from a file
    print("Model is loaded successfully !!!")
except:
    print("Model is not loaded successfully !!!")

#print(model.summary())

In [ ]:
pred_1 = model.predict(x) 

In [ ]:
print(pred_1[0])